Interger Sequence：
题目大意：给你一个整数序列，预测下一个值是多少？

1、通过前面序列，使用自回归模型得到参数，得到一个表达式，
d[n] = d[n-1] * a[0] + d[n-2] * a[1] + ... d[n-1 - p] * a[p];
但是自回归模型最大的问题，就是p是跟数据有关系的，一般通过观察得到p，例如求自协方差，偏协方差系数，观测等等，

我们这里可以假设p = 2,3,4,5...
怎么判断p的值？
例如：p=2
根据前几个数据得到参数，进行测试后几个值，如果误差在10^-2之内，我们就接受这个模型，并在所有序列中，去掉它，
在剩下的序列中，测试 p = 3,重复上面的操作，知道所有的序列都被满足，或者达到某种条件结束（防止此方法不能判断所有的序列），



In [85]:
import pandas as pd
import numpy as np
import csv
# 这里只用测试序列进行计算
testfile = '../data/test.csv'
resultfile = '../data/result.csv'

In [24]:

SequenceDict = {} #序列字典，<id,Sequence>

with open(testfile,'r') as f:
    reader = csv.reader(f)   
    i = 0
    for row in reader:
        if i == 0: i = 1;continue 
        
        
#         if i < 5:
#             print(row[0],np.array(row[1].split(',')).astype(int))
#         i += 1
        sequence = [int(x) for x in row[1].split(',')]
        SequenceDict[row[0]] = np.array(sequence)


d[n] = d[n-1] * a[0] + d[n-2] * a[1] + ... d[n-1 - p] * a[p];
可以转化为d[n] = d[0] * a[0] + d[1] * a[1] + ... d[p] * a[p]; 只是参数不同，功能相似

例如p = 2
[d0, d1, d2] [a0]    [d3]
[d1, d2, d3] [a1] =  [d4]
[d2, d3, d4] [a2]    [d5]
直接进行矩阵运算得到参数即可。
计算下面的矩阵得到参数即可。
[d0, d1, ..., dp]    [a0]    [d[p+1]]
[d1, d2, ..., dp+1]  [a1] =  [d[p+2]]
 .
 .
 .
[d[p-1], dp, ..., d[2p-1]]  [ap]    [d[2p]]

Xa = b

a = X-1 b



In [93]:
def autoRegression(sequence, p, k):
    '''
    sequence ： 序列
    p ：假设的参数个数
    coeff： 得到的参数
    
    检查p是否可以接受
    这里通过比较p之后的k个值，得到的预测值，是否与真实值的误差都在10^-2之内
    如果都满足就接受，否则就不接受,
    如果可以接受，就预测序列之后的值
    
    '''
    if len(sequence) < 2*p: return None
    
    x = []
    b = []
    for i in range(p):
        tmp = sequence[i:p+i]
        x.append(tmp)
    b = sequence[p:2*p]
    
    x = np.array(x)
    b = np.array(b)
    
    try: 
        if np.linalg.det(x)==0:
            return None
    except TypeError:
        return None
   
    coeff = np.linalg.inv(x).dot(b)
#     print(coeff)
    k = min(k, len(sequence) - 2*p)
    
    for i in range(k):
        t = 2*p + i
        predict = np.sum(coeff * sequence[t-p:t])
        if np.abs(predict - sequence[t]) > 10**-2:
            return None
    
    
    nextvalue = np.sum(sequence[len(sequence)-p:] * coeff)
    
    return int(round(nextvalue))
    

In [ ]:
#预测
predicted = dict()
k = 7
j = 0
for p in range(2,100): 
    i = 0
    for key in SequenceDict.keys():
        if key in predicted.keys(): continue
    
        sequence = np.array(SequenceDict[key])
#         print(sequence)
        nextvalue = autoRegression(sequence,p,k)
        
        if nextvalue != None:
            i += 1
            predicted[key] = nextvalue
    j += i
    print("p = ",p ,"i = ",i)
    
    if j == len(SequenceDict):
        break
    

In [107]:
print(len(SequenceDict))
len(predicted)

'''
填充没有预测到的序列，用这个序列的前一个替代它，当然这个可能不会对结果有什么好处，
但是，提交文件要求对所有的值，进行预测。
'''

result = []
for key in predicted.keys():
    result.append([int(key),predicted[key]])
for key in SequenceDict.keys():   
    if key not in predicted.keys():
        result.append([int(key),SequenceDict[key][-1]])

print(len(result))

113845
113845


In [108]:
sorted(result)

[[1, 587159944704],
 [2, 725161963867],
 [4, 158],
 [5, 3948],
 [6, 470],
 [9, 1461],
 [10, 6420255209482440],
 [12, 7564280930105061931496],
 [14, 10576607847561078075619087],
 [17, 2795903786354347468800],
 [19, 9418],
 [22, 32855719753],
 [25, 291],
 [27, -6777866008029815310109],
 [29, 1155],
 [30, 84],
 [31, 132],
 [32, -36312408],
 [33, -5120],
 [37, 5534],
 [38, 13995414281804368959953485044237],
 [39, 1234567],
 [42, 395505262],
 [43, 9113030337194775911093],
 [45, -5],
 [47, 91959],
 [48, 16782],
 [50, 6899124496],
 [54, -1],
 [55, 139],
 [56, 377599345119560766534],
 [58, 158],
 [60, 766582546402667648],
 [62, 2],
 [68, 462],
 [70, -297594848623],
 [71, 267],
 [74, 25958],
 [78, 6606477068251392],
 [79, 156],
 [80, 853498762909],
 [81, 610975075883240084201],
 [83, 6],
 [84, 87960930222080],
 [85, -26],
 [87, 82210569640327072],
 [88, -86],
 [90, 96147057896403],
 [93, 261640888432162766848],
 [96, 137],
 [97, -1760078],
 [98, 3],
 [99, 4722366482869645213696],
 [101, 2856800

In [109]:
with open(resultfile,'w') as f:
    writer = csv.writer(f)
    writer.writerow(['id','Last'])
    
    for data in result:
        writer.writerow([data[0],data[1]])
    
